In [14]:
import pandas as pd
import osmnx as ox
import networkx as nx
import random
import math
import folium
import os
import time
import numpy as np

In [15]:
# --- KONFIGURASI ---
filename = '../Dataset/jne_solo.csv'    # Pastikan file ini ada
nama_folder_output = "../Result"        # Folder output
KAPASITAS_TRUK = 200                    # Kapasitas Truk Canter (Paket)
TARGET_TOTAL_PAKET = 400                # Total paket dummy

# KOORDINAT HARDCODE JNE WAREHOUSE ADI SUMARMO (COLOMADU)
# (Sekitar Jl. Adi Sumarmo dekat Exit Tol/Bandara)
FIXED_WAREHOUSE_LAT = -7.532519372469423 
FIXED_WAREHOUSE_LON = 110.76423789494254 

print("✅ Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).")

✅ Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).


In [16]:
# ==========================================
# LOAD DATA & GENERATE DUMMY PAKET
# ==========================================
# 1. LOAD DATA AGEN
print("\n[Loading Data Agen...]")
try:
    df_jne = pd.read_csv(filename, sep=';')
    df_jne[['lat', 'lon']] = df_jne['Koordinat'].str.split(',', expand=True).astype(float)
    
    # Override lokasi Warehouse dengan input manual (Adi Sumarmo)
    warehouse_lat = FIXED_WAREHOUSE_LAT
    warehouse_lon = FIXED_WAREHOUSE_LON
    
    print(f"✅ {len(df_jne)} Agen Loaded.")
    print(f"📍 Warehouse FIXED: JNE Adi Sumarmo ({warehouse_lat}, {warehouse_lon})")

except Exception as e:
    print(f"❌ Error: {e}")

# 2. GENERATE DUMMY PAKET
print("\n[Generating Dummy Paket...]")
all_paket_data = []
paket_per_agen = TARGET_TOTAL_PAKET // len(df_jne) 
sisa_paket = TARGET_TOTAL_PAKET % len(df_jne)
counter_id = 1

def get_random_point_around(lat, lon, radius_km):
    r = radius_km / 111.0
    u = random.random()
    v = random.random()
    w = r * math.sqrt(u)
    t = 2 * math.pi * v
    x = w * math.cos(t)
    y_offset = w * math.sin(t)
    return lat + y_offset, lon + (x / math.cos(math.radians(lat)))

for idx, row in df_jne.iterrows():
    jatah = paket_per_agen + (1 if idx < sisa_paket else 0)
    for _ in range(jatah):
        new_lat, new_lon = get_random_point_around(row['lat'], row['lon'], 2.5)
        all_paket_data.append({
            'ID Paket': f'PKT-{counter_id:04d}',
            'lat': new_lat,
            'lon': new_lon,
            'Agen Induk': row['Nama JNE'],
            'Agen Lat': row['lat'],
            'Agen Lon': row['lon']
        })
        counter_id += 1

df_paket = pd.DataFrame(all_paket_data)
print(f"✅ Cell 2: {len(df_paket)} Paket Generated.")


[Loading Data Agen...]
✅ 48 Agen Loaded.
📍 Warehouse FIXED: JNE Adi Sumarmo (-7.532519372469423, 110.76423789494254)

[Generating Dummy Paket...]
✅ Cell 2: 400 Paket Generated.


In [17]:
# ==========================================
# 3. DOWNLOAD PETA (FULL KARESIDENAN SURAKARTA)
# ==========================================
print("\n[Downloading Map: Karesidenan Surakarta...]")
wilayah_solo_raya = [
    "Surakarta, Central Java, Indonesia",
    "Sukoharjo, Central Java, Indonesia",
    "Karanganyar, Central Java, Indonesia", 
    "Boyolali, Central Java, Indonesia",
    "Klaten, Central Java, Indonesia",
    "Sragen, Central Java, Indonesia",
    "Wonogiri, Central Java, Indonesia"
]

try:
    # Download graph gabungan
    G = ox.graph_from_place(wilayah_solo_raya, network_type='drive')
    print(f"✅ Cell 3: Peta Berhasil Didownload! ({len(G.nodes)} Simpul)")
except Exception as e:
    print(f"❌ Gagal download peta: {e}")


[Downloading Map: Karesidenan Surakarta...]
✅ Cell 3: Peta Berhasil Didownload! (322827 Simpul)


In [23]:
# ==========================================
# 4. HITUNG RUTE (REAL ROAD DISTANCE)
# ==========================================
print("\n[Cell 4: Menghitung Rute dengan Perekaman Urutan...]")

# --- A. RUTE TRUK (CVRP: Farthest First) ---
print("👉 Menghitung Rute Truk & Mencatat Urutan...")
truk_trips = []
visited_agents_per_trip = [] # <--- LIST BARU: Nyimpen urutan agen per trip
node_warehouse = ox.nearest_nodes(G, FIXED_WAREHOUSE_LON, FIXED_WAREHOUSE_LAT)

demand_per_agen = df_paket['Agen Induk'].value_counts().to_dict()
agen_to_visit = []
for idx, row in df_jne.iterrows():
    node = ox.nearest_nodes(G, row['lon'], row['lat'])
    demand = demand_per_agen.get(row['Nama JNE'], 0)
    if demand > 0:
        agen_to_visit.append({'Nama': row['Nama JNE'], 'Node': node, 'Demand': demand, 'lat': row['lat'], 'lon': row['lon']})

current_capacity = KAPASITAS_TRUK
current_node = node_warehouse
unvisited_agen = agen_to_visit.copy()
path_nodes_truk = [node_warehouse]

# Variabel pencatat urutan sementara
current_trip_agents = [] 
visit_order = 1 

while unvisited_agen:
    target_agen = None
    idx_remove = -1
    is_at_warehouse = (current_node == node_warehouse)
    best_metric_dist = -1.0 if is_at_warehouse else float('inf')

    for i, agen in enumerate(unvisited_agen):
        if agen['Demand'] <= current_capacity:
            try:
                dist = nx.shortest_path_length(G, current_node, agen['Node'], weight='length')
                # Logika: Di Gudang cari terjauh, Di Jalan cari terdekat
                if is_at_warehouse:
                    if dist > best_metric_dist:
                        best_metric_dist = dist; target_agen = agen; idx_remove = i
                else:
                    if dist < best_metric_dist:
                        best_metric_dist = dist; target_agen = agen; idx_remove = i
            except: continue

    if target_agen:
        path_nodes_truk.append(target_agen['Node'])
        current_node = target_agen['Node']
        current_capacity -= target_agen['Demand']
        unvisited_agen.pop(idx_remove)
        
        # REKAM DATA KUNJUNGAN
        current_trip_agents.append({
            'name': target_agen['Nama'],
            'lat': target_agen['lat'],
            'lon': target_agen['lon'],
            'order': visit_order, # <--- Ini nomor urutnya
            'demand': target_agen['Demand']
        })
        visit_order += 1
        print(f"   🚛 Urutan {visit_order-1}: {target_agen['Nama']}")
    else:
        # Balik Warehouse -> Simpan Data Trip Ini
        path_nodes_truk.append(node_warehouse)
        current_node = node_warehouse
        current_capacity = KAPASITAS_TRUK
        
        visited_agents_per_trip.append(current_trip_agents) # Simpan ke list utama
        current_trip_agents = [] # Reset
        visit_order = 1 # Reset urutan
        print("   🔄 Ganti Kloter (Trip Baru)")

if path_nodes_truk[-1] != node_warehouse:
    path_nodes_truk.append(node_warehouse)
    visited_agents_per_trip.append(current_trip_agents)

# Generate Geometri Truk
current_trip_segments = []
truk_trips = []
for k in range(len(path_nodes_truk) - 1):
    u = path_nodes_truk[k]; v = path_nodes_truk[k+1]
    try:
        seg = nx.shortest_path(G, u, v, weight='length')
        if k > 0 and len(current_trip_segments) > 0: seg = seg[1:]
        current_trip_segments.extend(seg)
        if v == node_warehouse:
            truk_trips.append(current_trip_segments)
            current_trip_segments = []
    except: pass

# --- B. RUTE KURIR (TSP) ---
print("\n👉 Menghitung Rute Kurir & Mencatat Urutan...")
rute_kurir_final = {}
paket_orders = {} # <--- LIST BARU: Nyimpen urutan paket

grouped = df_paket.groupby('Agen Induk')
for i, (nama, group) in enumerate(grouped):
    node_agen = ox.nearest_nodes(G, group.iloc[0]['Agen Lon'], group.iloc[0]['Agen Lat'])
    unvisited_packets = []
    for _, r in group.iterrows():
        unvisited_packets.append({
            'node': ox.nearest_nodes(G, r['lon'], r['lat']),
            'id': r['ID Paket'], 'lat': r['lat'], 'lon': r['lon']
        })

    curr = node_agen; path_seq = [node_agen]
    agent_paket_orders = []; order = 1
    
    while unvisited_packets:
        nearest_pkt = None; min_d = float('inf'); idx_remove = -1
        for idx, pkt in enumerate(unvisited_packets):
            try:
                d = nx.shortest_path_length(G, curr, pkt['node'], weight='length')
                if d < min_d: min_d = d; nearest_pkt = pkt; idx_remove = idx
            except: continue
            
        if nearest_pkt:
            path_seq.append(nearest_pkt['node'])
            curr = nearest_pkt['node']
            unvisited_packets.pop(idx_remove)
            
            # REKAM URUTAN PAKET
            agent_paket_orders.append({
                'id': nearest_pkt['id'],
                'order': order, # <--- Nomor urut paket
                'lat': nearest_pkt['lat'],
                'lon': nearest_pkt['lon']
            })
            order += 1
        else: break
            
    path_seq.append(node_agen)
    paket_orders[nama] = agent_paket_orders # Simpan

    # Geometri Kurir
    full_path = []
    for k in range(len(path_seq)-1):
        try:
            seg = nx.shortest_path(G, path_seq[k], path_seq[k+1], weight='length')
            if k > 0: seg = seg[1:]
            full_path.extend(seg)
        except: pass
    rute_kurir_final[nama] = full_path

print("✅ Cell 4: Selesai (Data Urutan Terekam).")


[Cell 4: Menghitung Rute dengan Perekaman Urutan...]
👉 Menghitung Rute Truk & Mencatat Urutan...
   🚛 Urutan 1: JNE Agen Bersaudara
   🚛 Urutan 2: Agen JNE Kumbolo
   🚛 Urutan 3: JNE 048 Papahan
   🚛 Urutan 4: Cabang JNE Karanganyar
   🚛 Urutan 5: JNE Mayor Ahmadi
   🚛 Urutan 6: JNE Agen 075
   🚛 Urutan 7: JNE Mojolaban
   🚛 Urutan 8: JNE Agen 23 Pasar Kliwon
   🚛 Urutan 9: JNE Pasar Klewer
   🚛 Urutan 10: JNE Agen Gading
   🚛 Urutan 11: JNE Agen 064
   🚛 Urutan 12: JNE Citra Damai
   🚛 Urutan 13: JNE Gubug Mas 016
   🚛 Urutan 14: JNE Express Slamet Riyadi
   🚛 Urutan 15: JNE Express 031
   🚛 Urutan 16: JNE Cantika
   🚛 Urutan 17: JNE Gloria
   🚛 Urutan 18: JNE Akasia
   🚛 Urutan 19: JNE AJS Ngemplak
   🚛 Urutan 20: JNE Kandang Sapi
   🚛 Urutan 21: JNE Suha Sutami
   🚛 Urutan 22: JNE Express Agen 033
   🚛 Urutan 23: JNE Arlita Gilingan
   🔄 Ganti Kloter (Trip Baru)
   🚛 Urutan 1: JNE Sukoharjo
   🚛 Urutan 2: JNE Agen Diva 073
   🚛 Urutan 3: JNE Solobaru
   🚛 Urutan 4: JNE Virgo / Tipe

In [24]:
# ==========================================
# 5. VISUALISASI FOLIUM
# ==========================================
from folium.features import DivIcon # Wajib import ini

print("\n[Cell 5: Visualisasi Marker Angka...]")
m = folium.Map(location=[FIXED_WAREHOUSE_LAT, FIXED_WAREHOUSE_LON], zoom_start=11, tiles='OpenStreetMap')

# --- LAYER 1: TRUK (Kloter 1 & 2) ---
layer_wh = folium.FeatureGroup(name="🚚 Jalur Truk (CVRP)", show=True)
folium.Marker(
    [FIXED_WAREHOUSE_LAT, FIXED_WAREHOUSE_LON],
    popup="<b>WAREHOUSE PUSAT</b>",
    icon=folium.Icon(color='black', icon='plane', prefix='fa')
).add_to(layer_wh)

# Konfigurasi Warna: Cuma Hijau & Merah
trip_colors = ['green', 'red'] 

for i, trip_nodes in enumerate(truk_trips):
    # Pilih warna: Genap=Hijau, Ganjil=Merah (atau sebaliknya)
    warna = trip_colors[i % 2]
    nama_trip = f"Truk Kloter #{i+1}"

    # 1. Gambar Garis Jalan
    if trip_nodes:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in trip_nodes]
        folium.PolyLine(coords, color=warna, weight=5, opacity=0.8, tooltip=nama_trip).add_to(layer_wh)

    # 2. Gambar Marker Angka (Agen)
    # Ambil data urutan yang udah kita rekam di Cell 4
    if i < len(visited_agents_per_trip):
        for agent_info in visited_agents_per_trip[i]:
            # Bikin Ikon Bulat ada Angkanya
            icon_html = f"""
                <div style="
                    font-size: 12pt; color: white; background-color: {warna};
                    border: 2px solid white; border-radius: 50%;
                    text-align: center; line-height: 30px; 
                    width: 30px; height: 30px;
                    box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
                    font-weight: bold;">
                    {agent_info['order']}
                </div>
            """
            
            folium.Marker(
                location=[agent_info['lat'], agent_info['lon']],
                popup=f"<b>{agent_info['name']}</b><br>Kloter: {i+1}<br>Urutan: {agent_info['order']}",
                tooltip=f"Urutan {agent_info['order']} ({agent_info['name']})",
                icon=DivIcon(icon_size=(30,30), icon_anchor=(15,15), html=icon_html)
            ).add_to(layer_wh)

layer_wh.add_to(m)

# --- LAYER 2: KURIR (Marker Angka Kecil) ---
for nama, nodes in rute_kurir_final.items():
    layer_kurir = folium.FeatureGroup(name=f"🛵 Area: {nama}", show=False)

    if nodes:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in nodes]
        folium.PolyLine(coords, color='#32CD32', weight=3, opacity=0.8).add_to(layer_kurir)
        
        # Marker Pangkalan Kurir (Icon Motor)
        start = nodes[0]
        folium.Marker(
            [G.nodes[start]['y'], G.nodes[start]['x']],
            popup=f"Start Kurir: {nama}",
            icon=folium.Icon(color='blue', icon='motorcycle', prefix='fa')
        ).add_to(layer_kurir)

    # Marker Paket (Angka Kecil)
    if nama in paket_orders:
        for pkg_info in paket_orders[nama]:
            icon_html_pkg = f"""
                <div style="
                    font-size: 9pt; color: white; background-color: #32CD32;
                    border: 1px solid white; border-radius: 50%;
                    text-align: center; line-height: 20px; 
                    width: 20px; height: 20px;">
                    {pkg_info['order']}
                </div>
            """
            
            folium.Marker(
                location=[pkg_info['lat'], pkg_info['lon']],
                popup=f"Paket: {pkg_info['id']}<br>Antrian: {pkg_info['order']}",
                icon=DivIcon(icon_size=(20,20), icon_anchor=(10,10), html=icon_html_pkg)
            ).add_to(layer_kurir)

    layer_kurir.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
print("✅ Cell 5: Peta dengan Marker Angka Siap!")


[Cell 5: Visualisasi Marker Angka...]
✅ Cell 5: Peta dengan Marker Angka Siap!


In [25]:
# ==========================================
# 6. SIMPAN OUTPUT
# ==========================================
# Setup Folder
base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
output_dir = os.path.join(base_dir, nama_folder_output)
if not os.path.exists(output_dir): os.makedirs(output_dir)

# Save HTML
html_path = os.path.join(output_dir, "peta_distribusi_CVRP_Real.html")
m.save(html_path)

print(f"✅ Cell 6: File saved at: {html_path}")

✅ Cell 6: File saved at: d:\Kuliah\Semester 3\Mat Diskrit\Project\Pembagian-Area-Pengiriman-JNE\Src\../Result\peta_distribusi_CVRP_Real.html
